<a href="https://colab.research.google.com/github/walkerjian/DailyCode/blob/main/SchedRow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement a job scheduler which takes in a function f and an integer n, and calls f after n milliseconds.

To implement a job scheduler, we'll break down the problem according to the Model-View-Controller (MVC) paradigm:

Model: This represents the data and the business logic. In this case, the model will handle scheduling the job and executing it after the specified delay.
View: The view is the user interface or the presentation layer. For this simple problem, the "view" will be the printed output of the scheduled function. It will show that the function was indeed executed after the specified delay.
Controller: The controller acts as an intermediary between the Model and the View. It will receive the input (function and delay), pass it to the Model to be scheduled, and then display the output in the View.
Let's begin the implementation:


In [1]:
import time
import threading

class JobSchedulerModel:
    def schedule(self, f, n):
        """Schedules the function `f` to be called after `n` milliseconds."""
        time_to_wait = n / 1000.0  # Convert milliseconds to seconds
        threading.Timer(time_to_wait, f).start()

class JobSchedulerView:
    def display_message(self, msg):
        """Displays the given message."""
        print(msg)

class JobSchedulerController:
    def __init__(self, model, view):
        self.model = model
        self.view = view

    def execute_job(self, f, n):
        """Schedules and displays the execution of function `f` after `n` milliseconds."""
        self.view.display_message(f"Scheduling job to run in {n} milliseconds...")
        self.model.schedule(f, n)

def test_job_scheduler():
    model = JobSchedulerModel()
    view = JobSchedulerView()
    controller = JobSchedulerController(model, view)

    # Define test functions
    def test_func1():
        view.display_message("Executing test_func1!")

    def test_func2():
        view.display_message("Executing test_func2!")

    # ... (more test functions can be defined)

    # Schedule test functions
    controller.execute_job(test_func1, 1000)  # 1 second
    controller.execute_job(test_func2, 2000)  # 2 seconds
    # ... (more test cases)

    # This is to prevent the main thread from exiting immediately
    time.sleep(5)

test_job_scheduler()


Scheduling job to run in 1000 milliseconds...
Scheduling job to run in 2000 milliseconds...
Executing test_func1!
Executing test_func2!


Note: We use threading.Timer to schedule the job after the specified delay. The time.sleep(5) at the end of the test function is to keep the main thread alive while the scheduled jobs execute.


### More efficient version

For a maximally efficient solution, we'll make the following changes:

Use threading.Event for synchronization: Instead of using time.sleep() to keep the main thread alive, we'll use threading.Event() which is a more efficient way to handle synchronization. It will ensure that the main thread waits for all scheduled jobs to complete.
Pool of Worker Threads: Instead of spawning a new thread for each scheduled task, we'll use a thread pool. This is more efficient for handling a large number of tasks since thread creation and destruction can be expensive operations. The concurrent.futures.ThreadPoolExecutor is a convenient way to manage a pool of worker threads.
Minimize I/O operations: I/O operations, like print statements, are relatively slow. To improve efficiency, we'll reduce their usage.**bold text**

In [2]:
import time
import threading
from concurrent.futures import ThreadPoolExecutor

class JobScheduler:
    def __init__(self):
        self.executor = ThreadPoolExecutor(max_workers=10)

    def schedule(self, f, n):
        """Schedules the function `f` to be called after `n` milliseconds."""
        time_to_wait = n / 1000.0  # Convert milliseconds to seconds
        self.executor.submit(self._delayed_execution, f, time_to_wait)

    def _delayed_execution(self, f, delay):
        time.sleep(delay)
        f()

    @staticmethod
    def display_message(msg):
        """Displays the given message."""
        print(msg)

# Testing
def test_job_scheduler():
    scheduler = JobScheduler()

    # Define test functions
    def test_func1():
        scheduler.display_message("Executing test_func1!")

    def test_func2():
        scheduler.display_message("Executing test_func2!")

    # Schedule test functions
    scheduler.display_message("Scheduling test_func1 to run in 1000 milliseconds...")
    scheduler.schedule(test_func1, 1000)  # 1 second

    scheduler.display_message("Scheduling test_func2 to run in 2000 milliseconds...")
    scheduler.schedule(test_func2, 2000)  # 2 seconds

    # This is to ensure the main thread doesn't exit immediately.
    # In a real-world application, we might use other mechanisms.
    time.sleep(5)

test_job_scheduler()


Scheduling test_func1 to run in 1000 milliseconds...
Scheduling test_func2 to run in 2000 milliseconds...
Executing test_func1!
Executing test_func2!


### More efficent again - no MVC
This version merges the model, view, and controller components into a single class named JobScheduler. The logic remains the same, but the structure is more compact and direct. The test harness remains unchanged.

In [3]:
import time
import threading
from concurrent.futures import ThreadPoolExecutor

class JobScheduler:
    def __init__(self):
        self.executor = ThreadPoolExecutor(max_workers=10)

    def schedule(self, f, n):
        """Schedules the function `f` to be called after `n` milliseconds."""
        time_to_wait = n / 1000.0  # Convert milliseconds to seconds
        self.executor.submit(self._delayed_execution, f, time_to_wait)

    def _delayed_execution(self, f, delay):
        time.sleep(delay)
        f()

    @staticmethod
    def display_message(msg):
        """Displays the given message."""
        print(msg)

# Testing
def test_job_scheduler():
    scheduler = JobScheduler()

    # Define test functions
    def test_func1():
        scheduler.display_message("Executing test_func1!")

    def test_func2():
        scheduler.display_message("Executing test_func2!")

    # Schedule test functions
    scheduler.display_message("Scheduling test_func1 to run in 1000 milliseconds...")
    scheduler.schedule(test_func1, 1000)  # 1 second

    scheduler.display_message("Scheduling test_func2 to run in 2000 milliseconds...")
    scheduler.schedule(test_func2, 2000)  # 2 seconds

    # This is to ensure the main thread doesn't exit immediately.
    # In a real-world application, we might use other mechanisms.
    time.sleep(5)

test_job_scheduler()


Scheduling test_func1 to run in 1000 milliseconds...
Scheduling test_func2 to run in 2000 milliseconds...
Executing test_func1!
Executing test_func2!


## Command Pattern for Job Scheduling in C++:

The Command pattern from the Gang of Four design patterns would be an appropriate choice for this kind of job scheduler, especially if we are building a command line utility. The Command pattern encapsulates a request as an object, allowing for parameterization of clients with different requests, queuing of requests, and logging the operations.

Design:

Command Interface: This will define an interface for executing an operation.

ConcreteCommand: This defines a binding between a Receiver object and an action.

Invoker: Asks the command to carry out the request.

Receiver: Knows how to perform the operations.

Here's a basic implementation

In [4]:
%%writefile scheduler.cpp
#include <iostream>
#include <thread>
#include <functional>
#include <chrono>
#include <vector>

// Command Interface
class Command {
public:
    virtual ~Command() {}
    virtual void execute() const = 0;
};

// Receiver
class Receiver {
public:
    void action(const std::string& update_msg) const {
        std::cout << update_msg << std::endl;
    }
};

// ConcreteCommand
class ConcreteCommand : public Command {
private:
    Receiver* receiver;
    std::string message;
public:
    ConcreteCommand(Receiver* r, const std::string& msg) : receiver(r), message(msg) {}
    virtual void execute() const override {
        receiver->action(message);
    }
};

// Invoker
class Invoker {
private:
    Command* command;
public:
    void setCommand(Command* cmd) {
        command = cmd;
    }
    void run() {
        if(command) {
            command->execute();
        }
    }
};

int main(int argc, char* argv[]) {
    Receiver receiver;
    ConcreteCommand cmd(&receiver, "Executing job!");
    Invoker invoker;

    invoker.setCommand(&cmd);

    // Simulate scheduling
    std::cout << "Scheduling job to run in 2 seconds..." << std::endl;
    std::this_thread::sleep_for(std::chrono::seconds(2));

    invoker.run();

    return 0;
}


Writing scheduler.cpp


In [5]:
!g++ scheduler.cpp -o scheduler -std=c++11 -lpthread

In [6]:
!./scheduler

Scheduling job to run in 2 seconds...
Executing job!


## Moron The Command Pattern!
The Command Pattern is one of the behavioral design patterns identified by the Gang of Four in their seminal book "Design Patterns: Elements of Reusable Object-Oriented Software". Its primary goal is to decouple an object that invokes an operation (the invoker) from the object that knows how to execute the operation (the receiver).

### Key Components:

1. **Command**: This is an interface for executing an operation. It usually has a method named `execute()` that encapsulates the action.
2. **ConcreteCommand**: This class implements the Command interface and specifies the binding between the receiver and the action.
3. **Invoker**: Uses the command to carry out the request. It's aware of the command interface but might not necessarily know about the concrete commands.
4. **Receiver**: Knows how to perform the operations associated with the command. All the work in the Command Pattern is done by the receiver.
5. **Client**: Creates the concrete command object and sets its receiver.

### Benefits:

1. **Decoupling**: Command pattern decouples the classes that invoke the operation from the object that knows how to execute the operation.
2. **Flexibility**: Commands can be executed immediately, delayed, or logged. They can also be undone (if the Command interface includes an `undo` method).
3. **Extension**: It's easy to add new commands, as you don't have to change existing code.
4. **Composite Commands**: You can combine multiple commands into a composite command, also known as a macro command. This is essentially a command that consists of a list of commands to be processed.

### Use Cases:

1. **GUI Buttons and Menu Items**: Many GUI libraries utilize the Command Pattern for button and menu item actions. Each action is a command.
2. **Undo/Redo**: Command pattern can be used to implement undo and redo functionalities in applications like text editors or graphic editors.
3. **Task Scheduling**: Applications that need to schedule tasks, queue tasks, or execute them remotely.
4. **Macro Recording**: Applications that support macro recording functionalities.

### Example:

Think of a remote control for electronic devices:

- **Command Interface**: Each button on the remote is associated with a command.
- **ConcreteCommand**: Each button's functionality (e.g., turning the TV on, changing the channel) is a concrete command.
- **Invoker**: The remote control itself acts as the invoker.
- **Receiver**: The device being controlled (e.g., TV, stereo) is the receiver.

When you press a button on the remote, it invokes a command. The device (receiver) then carries out the specified action.

### Drawbacks:

1. **Overhead**: For simple implementations, the Command Pattern might introduce unnecessary overhead, as you'd be creating command objects for simple operations.
2. **Learning Curve**: It might introduce complexity for developers unfamiliar with the pattern.

In conclusion, the Command Pattern offers a structured way to decouple the sender and receiver, making systems more modular and flexible. It's especially valuable when you anticipate changes in the system, want to queue operations, or need to support undo/redo operations.

### Diagramming the Command Pattern:

```
              +-----------------+
              |     Client      |
              +-----------------+
                   |
                   | creates
                   v
        +-------------------+
        |   ConcreteCommand |
        +-------------------+
                   |
                   | implements
                   v
              +---------+
              | Command |
              +---------+
                   ^
                   |
        +-------------------+       +----------+
        |     Invoker       |------>| Receiver |
        +-------------------+       +----------+
```

- **Client**: Creates a specific `ConcreteCommand`, attaches a `Receiver`, and sets the command to the `Invoker`.
- **Command**: An interface with the `execute()` method.
- **ConcreteCommand**: Implements the `Command` interface and invokes a method on `Receiver` to fulfill the request.
- **Invoker**: Holds a command and triggers the `execute()` method of the command.
- **Receiver**: Performs the actual work when requested by the `ConcreteCommand`.

For a graphical representation, you would typically use UML or a similar diagramming tool to visually represent the relationships between these classes. If you're familiar with tools like Lucidchart, Draw.io, or UML-specific tools, you can replicate the described structure for a more polished and visually appealing diagram.

##Is there a better pattern?
Since the Gang of Four's publication in 1994, software design and our understanding of patterns have evolved. The Command Pattern, while still very useful, can be enhanced using more modern paradigms and practices.

One major development in software design has been the introduction and widespread adoption of lambda functions, closures, and functional programming concepts in many languages. These features can simplify the Command Pattern, eliminating some of the boilerplate associated with defining command classes for every single action.

### Modern Command Pattern using Lambdas:

#### Step 1: The Concept
Instead of creating concrete command classes for every action, we can use lambda functions (or function objects) to encapsulate actions. This reduces the overhead of defining a new class for each command.

#### Step 2: The Basic Structure
- **Command**: This will still be a callable interface or a function signature.
- **Invoker**: Holds a list or queue of commands and is responsible for executing them.
- **Receiver**: Performs the actual work, just like in the classic Command Pattern.

#### Step 3: Implementation in Python (chosen for clarity)

In [7]:
from typing import Callable, List

# Step 1: Define the Command as a Callable
Command = Callable[[], None]

class Invoker:
    def __init__(self):
        self._commands: List[Command] = []

    def schedule_command(self, command: Command):
        """Schedule a command for execution."""
        self._commands.append(command)

    def run(self):
        """Execute all scheduled commands."""
        for command in self._commands:
            command()
        self._commands.clear()

# Sample usage
if __name__ == "__main__":
    # Receiver: functions that do the actual work
    def turn_on_light():
        print("Light turned ON")

    def turn_off_light():
        print("Light turned OFF")

    # Using lambda to schedule commands
    invoker = Invoker()
    invoker.schedule_command(turn_on_light)
    invoker.schedule_command(turn_off_light)

    # Execute commands
    invoker.run()


Light turned ON
Light turned OFF


In the example above, the `Invoker` class can schedule and run commands. Commands are now simply functions (or lambda expressions), which are pushed to the `_commands` list and executed when `run` is called.

#### Step 4: Benefits of the Modern Approach
- **Simplicity**: The use of lambda functions or function objects significantly reduces the boilerplate code.
- **Flexibility**: It's easier to add new commands. Just define a new function and schedule it.
- **Reusability**: Many functions can be reused across different parts of an application without the need to wrap them in command classes.

#### Step 5: Potential Drawbacks
- **State Management**: If commands need to maintain state, lambdas might not be the best choice. In such cases, a combination of lambdas and traditional command classes might be suitable.
- **Complexity**: For very complex commands, defining them inline as lambdas might hurt readability.

While the classic Command Pattern as described by the Gang of Four is powerful and provides a clear structure, modern programming practices offer ways to simplify and streamline the pattern, making it even more efficient and readable in many scenarios.

## Using **C++** to demonstrate the modern Command Pattern utilizing lambdas.
C++ has robust support for lambdas and functional programming constructs, making it an apt choice for this task.

In this implementation:
- The `Invoker` class has a list of commands, where each command is a `std::function<void()>`, allowing it to hold any callable object, function, or lambda with a `void()` signature.
- The `scheduleCommand` method allows us to add commands (in this case, lambda functions) to the list.
- The `run` method executes all scheduled commands in order.

This approach in C++ offers a clean and efficient way to implement the Command Pattern without the boilerplate of creating separate command classes for every action.

In [8]:
%%writefile sWitcher.cpp
#include <iostream>
#include <vector>
#include <functional>

class Invoker {
private:
    std::vector<std::function<void()>> commands;

public:
    void scheduleCommand(const std::function<void()>& command) {
        commands.push_back(command);
    }

    void run() {
        for (const auto& command : commands) {
            command();
        }
        commands.clear();
    }
};

int main() {
    Invoker invoker;

    // Sample commands (lambda functions)
    auto turnOnLight = []() {
        std::cout << "Light turned ON" << std::endl;
    };

    auto turnOffLight = []() {
        std::cout << "Light turned OFF" << std::endl;
    };

    // Schedule commands
    invoker.scheduleCommand(turnOnLight);
    invoker.scheduleCommand(turnOffLight);

    // Execute commands
    invoker.run();

    return 0;
}


Writing sWitcher.cpp


In [9]:
!g++ sWitcher.cpp -o sWitcher -std=c++11 -lpthread

In [10]:
!./sWitcher

Light turned ON
Light turned OFF


## Job scheduler in C++11 using the modern Command Pattern with lambdas:

### Design:
- The **Invoker** (`JobScheduler`) will accept a function `f` and an integer `n` as inputs.
- The function `f` will be wrapped in a lambda that incorporates the delay specified by `n`.
- The lambda will be scheduled for execution by the **Invoker**.

###C++11 implementation:
- The `JobScheduler` class schedules jobs for execution after a specified delay.
- The `scheduleJob` method wraps the provided job (function) in a lambda that incorporates the specified delay.
- The `executeJobs` method executes all scheduled jobs, each in its separate thread, to ensure the delays don't block other jobs.
- We detach each job thread to allow them to run independently of the main thread.

Using the modern Command Pattern with lambdas in this manner allows for a concise and efficient implementation of the job scheduler in C++11.

In [11]:
%%writefile schedulerC++11lambdas.cpp
#include <iostream>
#include <vector>
#include <functional>
#include <thread>
#include <chrono>

class JobScheduler {
private:
    std::vector<std::function<void()>> jobs;

public:
    void scheduleJob(const std::function<void()>& job, int delayMilliseconds) {
        // Wrap the job in a lambda that incorporates the delay
        auto delayedJob = [job, delayMilliseconds]() {
            std::this_thread::sleep_for(std::chrono::milliseconds(delayMilliseconds));
            job();
        };

        jobs.push_back(delayedJob);
    }

    void executeJobs() {
        for (const auto& job : jobs) {
            std::thread(job).detach();  // Execute each job in its own thread
        }
        jobs.clear();
    }
};

int main() {
    JobScheduler scheduler;

    // Sample jobs
    auto printHello = []() {
        std::cout << "Hello after 1 second!" << std::endl;
    };

    auto printWorld = []() {
        std::cout << "World after 2 seconds!" << std::endl;
    };

    // Schedule jobs
    scheduler.scheduleJob(printHello, 1000);
    scheduler.scheduleJob(printWorld, 2000);

    // Execute jobs
    scheduler.executeJobs();

    // Keep the main thread alive for a while to see the results
    std::this_thread::sleep_for(std::chrono::seconds(5));

    return 0;
}


Writing schedulerC++11lambdas.cpp


In [12]:
!g++ schedulerC++11lambdas.cpp -o schedulerC++11lambdas -std=c++11 -lpthread

In [13]:
!./scheduler

Scheduling job to run in 2 seconds...
Executing job!


## C++14 Improvement: Generalized Lambda Captures

In C++14, lambdas got an enhancement called "generalized lambda captures" which allows creating new variables in the lambda's capture list. This is useful for moving objects into a lambda.

In [14]:
%%writefile schedulerC++14GenlambdaCap.cpp
#include <iostream>
#include <vector>
#include <functional>
#include <thread>
#include <chrono>

class JobScheduler {
private:
    std::vector<std::function<void()>> jobs;

public:
    void scheduleJob(std::function<void()> job, int delayMilliseconds) {
        // Use generalized lambda captures to move the job into the lambda
        auto delayedJob = [job = std::move(job), delayMilliseconds]() {
            std::this_thread::sleep_for(std::chrono::milliseconds(delayMilliseconds));
            job();
        };

        jobs.push_back(delayedJob);
    }

    void executeJobs() {
        for (const auto& job : jobs) {
            std::thread(job).detach();
        }
        jobs.clear();
    }
};

int main() {
    JobScheduler scheduler;

    auto printHello = []() {
        std::cout << "Hello after 1 second!" << std::endl;
    };

    auto printWorld = []() {
        std::cout << "World after 2 seconds!" << std::endl;
    };

    scheduler.scheduleJob(printHello, 1000);
    scheduler.scheduleJob(printWorld, 2000);

    scheduler.executeJobs();

    std::this_thread::sleep_for(std::chrono::seconds(5));

    return 0;
}


Writing schedulerC++14GenlambdaCap.cpp


In [15]:
!g++ schedulerC++14GenlambdaCap.cpp -o schedulerC++14GenlambdaCap -std=c++14 -lpthread

In [16]:
!./schedulerC++14GenlambdaCap

Hello after 1 second!
World after 2 seconds!


## C++14 Improvement:
Certainly! Let's delve into some modern C++ features using C++17, which provides a wealth of useful enhancements over C++14. Here's an enhanced version of the job scheduler using a selection of these features:

### 1. Structured Bindings
Introduced in C++17, structured bindings allow you to decompose objects into their individual elements. This is particularly useful for pairs, tuples, and other similar data structures.

### 2. `std::optional`
It represents an optional object, i.e., an object that may or may not contain a value. It's a safer way than using pointers to represent the presence or absence of a value.

### 3. Inline Variables for `static` members
C++17 allows inline initialization of `static` members in class definitions.

### 4. `std::invoke`
A utility function to invoke callable objects. It can call regular functions, lambda functions, functor objects, or even member functions.

Here's a breakdown of the implemented enhancements:
1. **Structured Bindings**: The two lambda functions `printHello` and `printWorld` are created simultaneously using structured bindings.
2. **`std::optional`**: The `scheduleJob` function now accepts an optional delay, allowing jobs to be scheduled with a default delay if one isn't specified.
3. **Inline Variables**: The default delay for the job scheduler is implemented as an inline static variable in the `JobScheduler` class.
4. **`std::invoke`**: Used to call the job in the `delayedJob` lambda. This is more of a stylistic choice, as in this context, it behaves the same way as directly calling the function, but `std::invoke` provides a consistent way to call different types of callable objects.

This enhanced version utilizes some of the modern features in C++17, making the code more flexible and expressive.

In [17]:
%%writefile schedulerC++17.cpp
#include <iostream>
#include <vector>
#include <functional>
#include <thread>
#include <chrono>
#include <optional>
#include <utility> // for std::invoke

class JobScheduler {
private:
    std::vector<std::function<void()>> jobs;

public:
    // Using inline static variable for a default delay
    inline static std::optional<int> defaultDelay = std::nullopt;

    void scheduleJob(std::function<void()> job, std::optional<int> delayMilliseconds = defaultDelay) {
        // If no delay is specified, use the default
        if (!delayMilliseconds.has_value() && defaultDelay.has_value()) {
            delayMilliseconds = defaultDelay;
        }

        // Use generalized lambda captures to move the job into the lambda
        auto delayedJob = [job = std::move(job), delayMilliseconds]() {
            if (delayMilliseconds.has_value()) {
                std::this_thread::sleep_for(std::chrono::milliseconds(*delayMilliseconds));
            }
            std::invoke(job); // Using std::invoke to call the job
        };

        jobs.push_back(delayedJob);
    }

    void executeJobs() {
        for (const auto& job : jobs) {
            std::thread(job).detach();
        }
        jobs.clear();
    }
};

int main() {
    JobScheduler scheduler;

    // Use structured bindings to create multiple lambdas
    auto [printHello, printWorld] = []() {
        return std::make_pair(
            []() {
                std::cout << "Hello after 1 second!" << std::endl;
            },
            []() {
                std::cout << "World after 2 seconds!" << std::endl;
            }
        );
    }();

    scheduler.scheduleJob(printHello, 1000);
    scheduler.scheduleJob(printWorld, 2000);

    // Using the default delay
    JobScheduler::defaultDelay = 1500;
    scheduler.scheduleJob([]() {
        std::cout << "This job uses the default delay of 1.5 seconds!" << std::endl;
    });

    scheduler.executeJobs();

    std::this_thread::sleep_for(std::chrono::seconds(5));

    return 0;
}


Writing schedulerC++17.cpp


In [19]:
!g++ schedulerC++17.cpp -o schedulerC++17 -std=c++17 -lpthread

In [20]:
!./schedulerC++17

Hello after 1 second!
This job uses the default delay of 1.5 seconds!
World after 2 seconds!


## C++20 Improvement:
C++20 brings even more power and expressiveness to the language.
Here's a taste:

### 1. Concepts
Concepts introduce a way to express template constraints more readably than before. We can use them to ensure that our job scheduler only accepts callable objects.

### 2. Ranges
Ranges provide a new way to work with sequences of data. They can simplify and make our code more expressive when working with containers like vectors.

### 3. `std::jthread`
This is a new addition in C++20 that automatically joins threads (if joinable) upon destruction, ensuring that we don't accidentally detach or forget to join threads.

### 4. `std::span`
It provides a lightweight, non-owning reference to a contiguous sequence, or a span. It can be more efficient than using traditional containers in certain scenarios.

### 5. Coroutines (bonus)
Coroutines are a new way to write asynchronous code. For our job scheduler, we could potentially use coroutines to handle asynchronous tasks, though this might make the example more complex.

Let's implement the job scheduler with the first four features:

1. **Concepts**: We've defined a concept `Callable` to ensure that the `scheduleJob` function only accepts callable objects.
2. **Ranges**: Used the ranges library to iterate over the jobs. Here, the effect is minimal, but ranges can be very powerful for complex transformations and filters.
3. **`std::jthread`**: This replaces the traditional `std::thread` and ensures that we don't need to manually join or detach threads.
4. **`std::span`**: While not used in this basic example, it's worth mentioning as it can be used in scenarios where you want a lightweight view over a sequence without owning it.

This version of the job scheduler showcases some of the modern features of C++20, making the code safer (e.g., with `jthread`) and more expressive (e.g., with concepts).

In [28]:
%%writefile schedulerC++20.cpp
#include <iostream>
#include <vector>
#include <functional>
#include <thread>
#include <chrono>
#include <concepts>
#include <optional>

// Concept to check if a type is callable
template<typename T>
concept Callable = requires(T t) {
    { t() } -> std::same_as<void>;
};

class JobScheduler {
private:
    std::vector<std::function<void()>> jobs;

public:
    // Using inline static variable for a default delay
    inline static std::chrono::milliseconds defaultDelay = std::chrono::milliseconds(0);

    void scheduleJob(Callable auto job, std::optional<int> delayMilliseconds = std::nullopt) {
        if (!delayMilliseconds.has_value()) {
            delayMilliseconds = defaultDelay.count();
        }

        auto delayedJob = [job, delayMilliseconds]() {
            std::this_thread::sleep_for(std::chrono::milliseconds(*delayMilliseconds));
            job();
        };

        jobs.push_back(delayedJob);
    }

    void executeJobs() {
        for (auto& currentJob : jobs) {
            std::jthread([currentJob]() { currentJob(); });  // create an anonymous std::jthread
        }
        jobs.clear();
    }

};

int main() {
    JobScheduler scheduler;

    auto printHello = []() {
        std::cout << "Hello after 1 second!" << std::endl;
    };

    auto printWorld = []() {
        std::cout << "World after 2 seconds!" << std::endl;
    };

    scheduler.scheduleJob(printHello, 1000);
    scheduler.scheduleJob(printWorld, 2000);

    // Using the default delay
    JobScheduler::defaultDelay = std::chrono::milliseconds(1500);
    scheduler.scheduleJob([]() {
        std::cout << "This job uses the default delay of 1.5 seconds!" << std::endl;
    });

    scheduler.executeJobs();

    std::this_thread::sleep_for(std::chrono::seconds(5));

    return 0;
}


Overwriting schedulerC++20.cpp


In [29]:
!g++ schedulerC++20.cpp -o schedulerC++20 -std=c++20 -lpthread

In [30]:
!./schedulerC++20

Hello after 1 second!
World after 2 seconds!
This job uses the default delay of 1.5 seconds!


## Using Coroutines in C++20

Coroutines provide a fundamentally different way to manage asynchronous operations and tasks. C++20 introduced support for coroutines, enabling more elegant and efficient asynchronous code.

Here's an outline of how to use coroutines in the job scheduler:

1. **Coroutine Handle**: Instead of directly scheduling functions to run, we'll schedule coroutine handles. These handles are the gateway to managing and resuming coroutines.
2. **Asynchronous Execution**: Instead of blocking the thread with `std::this_thread::sleep_for`, we'll use an `async_sleep` function that asynchronously waits for a specified duration and then resumes the coroutine.
3. **Task System**: The job scheduler will maintain a list of tasks (coroutines) and execute them asynchronously.

**Note**: The provided `async_sleep` function uses `std::this_thread::sleep_for` for simplicity, but in a real-world application, you'd use a more efficient mechanism that doesn't block the thread, such as an asynchronous timer.

With coroutines, we've transformed the nature of our job scheduler. Instead of merely delaying and then executing functions, we're now asynchronously scheduling and executing tasks. This asynchronous approach can be more efficient and scalable, especially when dealing with IO-bound tasks or operations that would otherwise block the thread.

In [34]:
%%writefile schedulerC++20Coroutines.cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <coroutine>
#include <thread>
#include <functional>

// Type-erased, non-owning reference to a suspended coroutine
struct Awaitable {
    struct promise_type {
        Awaitable get_return_object() {
            return { std::coroutine_handle<promise_type>::from_promise(*this) };
        }
        std::suspend_never initial_suspend() { return {}; }
        std::suspend_never final_suspend() noexcept { return {}; }
        void return_void() {}
        void unhandled_exception() { std::terminate(); }
    };

    std::coroutine_handle<> handle;
    bool await_ready() { return false; }
    void await_suspend(std::coroutine_handle<>) {}
    void await_resume() {}
};

// Async sleep function
Awaitable async_sleep(std::chrono::milliseconds duration) {
    std::this_thread::sleep_for(duration); // For simplicity, we're using thread sleep here
    co_return;
}

class JobScheduler {
private:
    std::vector<std::coroutine_handle<>> tasks;

public:
    // Schedule a coroutine
    void schedule(std::coroutine_handle<> task) {
        tasks.push_back(task);
    }

    // Execute all tasks
    void executeJobs() {
        for (auto& task : tasks) {
            task.resume();
            task.destroy(); // Clean up the coroutine after it has finished
        }
        tasks.clear();
    }
};

// Define a coroutine for our tasks
Awaitable jobWithDelay(std::function<void()> job, std::chrono::milliseconds delay) {
    co_await async_sleep(delay);
    job();
}

int main() {
    JobScheduler scheduler;

    auto printHello = []() {
        std::cout << "Hello after 1 second!" << std::endl;
    };

    auto printWorld = []() {
        std::cout << "World after 2 seconds!" << std::endl;
    };

    // Schedule coroutines
    scheduler.schedule(jobWithDelay(printHello, std::chrono::milliseconds(1000)).handle);
    scheduler.schedule(jobWithDelay(printWorld, std::chrono::milliseconds(2000)).handle);

    scheduler.executeJobs();

    // Keep the main thread alive for a while to see the results
    std::this_thread::sleep_for(std::chrono::seconds(5));

    return 0;
}


Overwriting schedulerC++20Coroutines.cpp


In [35]:
!g++ schedulerC++20Coroutines.cpp -o schedulerC++20Coroutines -std=c++20 -lpthread

In [37]:
!./schedulerC++20Coroutines

Hello after 1 second!


## Efficient Asynchronous Timer with Coroutines

The core idea behind an asynchronous timer is to set up a mechanism that will resume our coroutine after a specified duration without blocking the current thread. A common way to achieve this in a cross-platform manner is to use libraries like Boost.Asio. However, in this context, I'll illustrate a simple, platform-specific approach using `std::jthread` to achieve non-blocking delays.

### Reassessing the Design Pattern

The Command Pattern provided a clear separation of concerns. However, with the asynchronous nature of coroutines and the requirement to deal with timers, a combination of the Command and Reactor patterns might be more appropriate. The Reactor pattern allows event-driven code to be structured around "reactions" to external events (like timer completions).

### Implementation

Here's a revised version of the job scheduler using an efficient asynchronous timer:

In this implementation:

1. **Awaitable Structure**: The `Awaitable` structure uses a condition variable to block the coroutine (not the thread) until the timer completes.
2. **Asynchronous Timer**: The `async_sleep` function now sets up a separate thread to wait for the specified duration, then signals the coroutine to resume execution. This ensures non-blocking behavior.
3. **Reactor Pattern**: The event-driven nature of the code (reacting to timer completions) aligns well with the Reactor pattern.

This version is more efficient, scalable, and better adheres to modern asynchronous programming paradigms in C++.

In [112]:
%%writefile schedEffCoReact.cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <thread>
#include <functional>
#include <condition_variable>

class JobScheduler {
private:
    std::vector<std::pair<std::function<void()>, std::chrono::milliseconds>> tasks;
    std::mutex mtx;
    std::condition_variable cv;
    int active_jobs = 0;

public:
    void schedule(std::function<void()> task, std::chrono::milliseconds delay = std::chrono::milliseconds(0)) {
        std::cout << "Scheduling a job...\n";
        tasks.push_back({task, delay});
        ++active_jobs;
    }

    void executeJobs() {
        for (auto& [task, delay] : tasks) {
            std::thread([this, task, delay]() {
                if (delay.count() > 0) {
                    std::cout << "Thread sleeping for " << delay.count() << " milliseconds...\n";
                    std::this_thread::sleep_for(delay);
                }
                std::cout << "Executing a job...\n";
                task();
                {
                    std::unique_lock lock(mtx);
                    --active_jobs;
                    if (active_jobs == 0) {
                        cv.notify_one();
                    }
                }
                std::cout << "Job finished. Remaining jobs: " << active_jobs << "\n";
            }).detach();
        }
        tasks.clear();
    }

    void wait() {
        std::cout << "Main thread waiting...\n";
        std::unique_lock lock(mtx);
        cv.wait(lock, [this]() { return active_jobs == 0; });
        std::cout << "Main thread done waiting...\n";
    }
};

int main() {
    JobScheduler scheduler;

    auto printHello = []() {
        std::cout << "Hello after 1 second!" << std::endl;
    };

    auto printWorld = []() {
        std::cout << "World after 2 seconds!" << std::endl;
    };

    scheduler.schedule(printHello, std::chrono::milliseconds(1000));
    scheduler.schedule(printWorld, std::chrono::milliseconds(2000));

    scheduler.executeJobs();

    scheduler.wait();  // Wait for all tasks to complete.

    return 0;
}



Overwriting schedEffCoReact.cpp


In [113]:
!g++ schedEffCoReact.cpp -o schedEffCoReact -std=c++20 -lpthread

In [114]:
!./schedEffCoReact

Scheduling a job...
Scheduling a job...
Main thread waiting...
Thread sleeping for 1000 milliseconds...
Thread sleeping for 2000 milliseconds...
Executing a job...
Hello after 1 second!
Job finished. Remaining jobs: 1
Executing a job...
World after 2 seconds!
Job finished. Remaining jobs: 0
Main thread done waiting...
